# Linear Regression Example

Let's walk through the steps of the official documentation example. Doing this will help your ability to read from the documentation, understand it, and then apply it to your own problems (the upcoming Consulting Project).

In [19]:
# Create and initiate the Spark Session

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [20]:
# Load the training data 

from pyspark.ml.regression import LinearRegression

training = spark.read.format("libsvm").load("resources/sample_linear_regression_data.txt")

Interesting! We haven't seen libsvm formats before. In fact the aren't very popular when working with datasets in Python, but the Spark Documentation makes use of them a lot because of their formatting. Let's see what the training data looks like:

In [21]:
# Lets see the training data

training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

This is the format that Spark expects. Two columns with the names "label" and "features". 

The "label" column then needs to have the numerical label, either a regression numerical value, or a numerical value that matches to a classification grouping. Later on we will talk about unsupervised learning algorithms that by their nature do not use or require a label.

The feature column has inside of it a vector of all the features that belong to that row. Usually what we end up doing is combining the various feature columns we have into a single 'features' column using the data transformations we've learned about.

Let's continue working through this simple example!

In [22]:
# Create an instance of the model

lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

# You could also pass in additional parameters for regularization, do the reading 
# in ISLR to fully understand that, after that its just some simple parameter calls.
# Check the documentation with Shift+Tab for more info!

In [23]:
# fit the model

lrmodel = lr.fit(training)

In [24]:
# Get the coefficients and intercept

print(f"coefficients : {str(lrmodel.coefficients)}")
print("\n")
print(f"Intercept : {str(lrmodel.intercept)}")

coefficients : [0.0073350710225801715,0.8313757584337543,-0.8095307954684084,2.441191686884721,0.5191713795290003,1.1534591903547016,-0.2989124112808717,-0.5128514186201779,-0.619712827067017,0.6956151804322931]


Intercept : 0.14228558260358093


In [25]:
# We can check the summary for even more info

trainingSummary = lrmodel.summary

In [26]:
# Get the residuals, RMSE and Adjusted R2

trainingSummary.residuals.show()

print(f"RMSE : {trainingSummary.rootMeanSquaredError}")
print(f"Adjusted R2 : {trainingSummary.r2adj}")

+-------------------+
|          residuals|
+-------------------+
|-11.011130022096554|
| 0.9236590911176538|
|-4.5957401897776675|
|  -20.4201774575836|
|-10.339160314788181|
|-5.9552091439610555|
|-10.726906349283922|
|  2.122807193191233|
|  4.077122222293811|
|-17.316168071241652|
| -4.593044343959059|
|  6.380476690746936|
| 11.320566035059846|
|-20.721971774534094|
| -2.736692773777401|
| -16.66886934252847|
|  8.242186378876315|
|-1.3723486332690233|
|-0.7060332131264666|
|-1.1591135969994064|
+-------------------+
only showing top 20 rows

RMSE : 10.16309157133015
Adjusted R2 : 0.007999162774081858


## Train/Test Splits

But wait! We've commited a big mistake, we never separated our data set into a training and test set. Instead we trained on ALL of the data, something we generally want to avoid doing. Read ISLR and check out the theory lecture for more info on this, but remember we won't get a fair evaluation of our model by judging how well it does again on the same data it was trained on!

Luckily Spark DataFrames have an almost too convienent method of splitting the data! Let's see it:

In [27]:
all_data = spark.read.format("libsvm").load("resources/sample_linear_regression_data.txt")

In [28]:
# Pass in the split between training/test as a list.
# No correct, but generally 70/30 or 60/40 splits are used. 
# Depending on how much data you have and how unbalanced it is.

train_data, test_data = all_data.randomSplit([0.7,0.3])

In [29]:
# Check the training data
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|
|-17.026492264209548|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
|-16.692207021311106|(10,[0,1,2,3,4,5,...|
+----------

In [30]:
# Check the test data 
test_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
| -15.86200932757056|(10,[0,1,2,3,4,5,...|
|-15.310980589416289|(10,[0,1,2,3,4,5,...|
|-13.976130931152703|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
|-12.558575788856189|(10,[0,1,2,3,4,5,...|
|-12.198096564661412|(10,[0,1,2,3,4,5,...|
|-12.130353212287929|(10,[0,1,2,3,4,5,...|
|-11.827072996392571|(10,[0,1,2,3,4,5,...|
|-11.039347808253828|(10,[0,1,2,3,4,5,...|
|-10.600130341909033|(10,[0,1,2,3,4,5,...|
| -10.57769830424322|(10,[0,1,2,3,4,5,...|
| -10.35591860037468|(10,[0,1,2,3,4,5,...|
| -8.822357870425154|(10,[0,1,2,3,4,5,...|
+----------

In [31]:
# We will isolate the test features

unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [32]:
# We will now fit the model on traininig data
correct_model = lr.fit(train_data)

In [33]:
# We will now use this model on test data to evaluate
test_results = correct_model.evaluate(test_data)

In [34]:
# Lets see residuals, RMSE and Adj R2

test_results.residuals.show()

print(f"RMSE : {test_results.rootMeanSquaredError}")
print(f"Adj R2: {test_results.r2adj}")

+-------------------+
|          residuals|
+-------------------+
| -22.94604314912743|
|-27.383962410906737|
|-22.429033348729337|
|-20.847740489486565|
|-19.303876832099668|
| -15.56755348961093|
|-19.199782817200703|
|-15.643224624427061|
|-12.236810078993956|
| -14.00219722178975|
| -17.52647464585355|
|-13.403896135190687|
|-13.311323312777825|
| -9.758043459944826|
|-15.126350655085702|
|   -7.4664715177211|
|-11.586703488196921|
| -9.644330663007207|
|-13.665906984395368|
| -9.384091317650947|
+-------------------+
only showing top 20 rows

RMSE : 10.252249810852387
Adj R2: -0.1219891758582079


Well that is nice, but realistically we will eventually want to test this model against unlabeled data, after all, that is the whole point of building the model in the first place. We can again do this with a convenient method call, in this case, transform(). Which was actually being called within the evaluate() method. Let's see it in action:

In [35]:
predictions = correct_model.transform(unlabeled_data)
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -0.5648409411955424|
|(10,[0,1,2,3,4,5,...|   4.434136474710663|
|(10,[0,1,2,3,4,5,...|  0.9966455845635322|
|(10,[0,1,2,3,4,5,...|  0.7902578736973518|
|(10,[0,1,2,3,4,5,...| -0.3634417832720512|
|(10,[0,1,2,3,4,5,...| -1.9266468672724142|
|(10,[0,1,2,3,4,5,...|  1.7711082462611958|
|(10,[0,1,2,3,4,5,...|-0.21878470314349835|
|(10,[0,1,2,3,4,5,...| -3.0741705104223325|
|(10,[0,1,2,3,4,5,...|0.026066290637047643|
|(10,[0,1,2,3,4,5,...|  3.7540330841506804|
|(10,[0,1,2,3,4,5,...|  0.8453203463344987|
|(10,[0,1,2,3,4,5,...|  1.1132267481164118|
|(10,[0,1,2,3,4,5,...|  -2.372309752343102|
|(10,[0,1,2,3,4,5,...|   3.299277658693131|
|(10,[0,1,2,3,4,5,...|  -3.572876290532727|
|(10,[0,1,2,3,4,5,...|  0.9865731462878875|
|(10,[0,1,2,3,4,5,...| -0.9333676412360141|
|(10,[0,1,2,3,4,5,...|  3.3099883840206874|
|(10,[0,1,2,3,4,5,...|  0.561733